# Clustering Crypto

In [1]:
!pip install -U altair

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
# Initial imports
import requests
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import preprocessing
import altair as alt

min_max_scaler = preprocessing.MinMaxScaler()

### Fetching Cryptocurrency Data

In [3]:
# Importing Data
file_path = Path("crypto_data.csv")
# Creating a DataFrame
crypto_df = pd.read_csv(file_path)
# Keeping only necessary columns
'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df.set_index("Unnamed: 0", inplace = True)
crypto_df.index.name = None
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


### Data Preprocessing

In [4]:
# Keeping only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df["IsTrading"] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [5]:
# Keeping only cryptocurrencies with a working algorithm
crypto_df['Algorithm'].isnull().values.any()

False

In [6]:
# Removing the "IsTrading" column
crypto_df.drop(['IsTrading'], axis=1, inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [7]:
# Removing rows with at least 1 null value
# Finding null values
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [8]:
# Removing rows with at least 1 null value, # Dropping rows where there are 'N/A' text values
crypto_df.dropna(inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [9]:
# Removing rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] > 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [10]:
# Storing the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = crypto_df[["CoinName"]]
coins_name.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [11]:
# Dropping the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop("CoinName", axis=1, inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [12]:
# Creating dummy variables for text features
LE = LabelEncoder()
X = crypto_df.copy()
X['Algorithm'] = LE.fit_transform(X['Algorithm']) 
X['ProofType'] = LE.fit_transform(X['ProofType']) 
X.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,52,15,4.199995e+01,42
404,52,15,1.055185e+09,532000000
1337,66,15,2.927942e+10,314159265359
BTC,47,12,1.792718e+07,21000000
ETH,20,12,1.076842e+08,0


In [13]:
# Standardizing data
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[ 3.76459118e-01  8.91356555e-01 -1.17108170e-01 -1.52870298e-01]
 [ 3.76459118e-01  8.91356555e-01 -9.39695522e-02 -1.45008997e-01]
 [ 1.21543803e+00  8.91356555e-01  5.24945609e-01  4.48942416e+00]
 [ 7.68237937e-02  1.67233875e-03 -1.16715055e-01 -1.52559984e-01]
 [-1.54120696e+00  1.67233875e-03 -1.14746818e-01 -1.52870298e-01]]


### Reducing Dimensions Using PCA

In [14]:
# Using PCA to reduce dimensions to 3 principal components
# Initializing PCS model
pca = PCA(n_components=3)

In [15]:
# Creating a DataFrame with the principal components data
# Getting two principal components for the data.
X_pca = pca.fit_transform(X_scaled)
pcs_df = pd.DataFrame(data= X_pca, index=X.index, columns = ["PC 1", "PC 2","PC 3"])
pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.417875,0.810296,0.372138
404,-0.396564,0.815135,0.373256
1337,3.124076,2.209780,0.504335
BTC,-0.192083,0.016266,-0.072910
ETH,-0.044116,-1.167492,1.012525
LTC,-0.217954,0.235824,-0.273789
DASH,-0.482771,1.336609,-0.110253
XMR,-0.001979,-1.518631,1.334156
ETC,-0.041907,-1.167016,1.012991
ZEC,-0.034542,-1.255502,1.092984


In [16]:
# Fetching the explained variance
pca.explained_variance_ratio_

array([0.4350467 , 0.28670411, 0.20538956])

## Sample Conclusions
 Defining three principal components, we have 0.93% of the information in the original dataset, so we can conclude that using three principal components is a better approach to reduce the dimensions in this case.

### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [17]:
inertia = []
k = list(range(1, 11))

# Calculating the inertia for the range of k values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

# Creating the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
alt.Chart(df_elbow).mark_line().encode(x='k', y='inertia').configure_axis(grid=False).properties(title='Elbow Curve', width=600)

alt.Chart(...)

Running K-Means with `k=<your best value for k here>`

In [18]:
# Initializing the K-Means model
def get_clusters(k, data):
# Initialize the K-Means model
    model = KMeans(n_clusters=k, random_state=0)
# Fitting the model
    model.fit(data)
# Predicting clusters
    predictions = model.predict(data)
# Creating a new DataFrame including predicted clusters and cryptocurrencies features
    data["class"] = model.labels_

    return data

In [19]:
# Looking for clusters the first best value of k
five_clusters = get_clusters(5, pcs_df)
five_clusters.head()

,PC 1,PC 2,PC 3,class
42,-0.417875,0.810296,0.372138,4
404,-0.396564,0.815135,0.373256,4
1337,3.124076,2.209780,0.504335,1
BTC,-0.192083,0.016266,-0.072910,4
ETH,-0.044116,-1.167492,1.012525,3


In [20]:
# Creating a new DataFrame named clustered_df, that includes the following columns "Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "PC 1", "PC 2", "PC 3", "CoinName", "Class". 
clustered_df = X.merge(five_clusters,how='outer', left_index=True, right_index=True)
clustered_df['CoinName'] = coins_name.values
reorder = ['Algorithm','ProofType','TotalCoinsMined','TotalCoinSupply','PC 1', 'PC 2', 'PC 3', 'CoinName', 'class']
clustered_df = clustered_df.reindex(columns=reorder)
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,52,15,4.199995e+01,42,-0.417875,0.810296,0.372138,42 Coin,4
404,52,15,1.055185e+09,532000000,-0.396564,0.815135,0.373256,404Coin,4
1337,66,15,2.927942e+10,314159265359,3.124076,2.209780,0.504335,EliteCoin,1
BTC,47,12,1.792718e+07,21000000,-0.192083,0.016266,-0.072910,Bitcoin,4
ETH,20,12,1.076842e+08,0,-0.044116,-1.167492,1.012525,Ethereum,3
LTC,52,12,6.303924e+07,84000000,-0.217954,0.235824,-0.273789,Litecoin,4
DASH,64,15,9.031294e+06,22000000,-0.482771,1.336609,-0.110253,Dash,4
XMR,12,12,1.720114e+07,0,-0.001979,-1.518631,1.334156,Monero,3
ETC,20,12,1.133597e+08,210000000,-0.041907,-1.167016,1.012991,Ethereum Classic,3
ZEC,18,12,7.383056e+06,21000000,-0.034542,-1.255502,1.092984,ZCash,3


### Visualizing Results

#### 3D-Scatter with Clusters

In [21]:
# Creating a 2D-Scatter with the PCA data and the clusters
alt.Chart(clustered_df).mark_circle(size=60).encode(
    x='PC 1', y='PC 2', color=alt.Color('class'),
    tooltip=['CoinName', 'Algorithm', 'TotalCoinsMined', 
             'TotalCoinSupply']).interactive().configure_axis(grid=False).properties(title='Clusters Visualization', width=600)

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [22]:
# Printing the total number of tradable cryptocurrencies
number_of_tradable_ccs = len(clustered_df.index)
print(f"The Number of Tradable Cryptocurrencies is {number_of_tradable_ccs}")

The Number of Tradable Cryptocurrencies is 532


#### Scatter Plot with Tradable Cryptocurrencies

In [23]:
# Scaling data to create the scatter plot
column_names_to_normalize = ['TotalCoinsMined', 'TotalCoinSupply'] 
x_data = clustered_df[column_names_to_normalize].values
x_data_scaled = min_max_scaler.fit_transform(x_data)
clustered_df_scaled = pd.DataFrame(x_data_scaled, columns=column_names_to_normalize, index = clustered_df.index)
clustered_df[column_names_to_normalize] = clustered_df_scaled
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,52,15,0.000000,4.200000e-11,-0.417875,0.810296,0.372138,42 Coin,4
404,52,15,0.001066,5.320000e-04,-0.396564,0.815135,0.373256,404Coin,4
1337,66,15,0.029576,3.141593e-01,3.124076,2.209780,0.504335,EliteCoin,1
BTC,47,12,0.000018,2.100000e-05,-0.192083,0.016266,-0.072910,Bitcoin,4
ETH,20,12,0.000109,0.000000e+00,-0.044116,-1.167492,1.012525,Ethereum,3
LTC,52,12,0.000064,8.400000e-05,-0.217954,0.235824,-0.273789,Litecoin,4
DASH,64,15,0.000009,2.200000e-05,-0.482771,1.336609,-0.110253,Dash,4
XMR,12,12,0.000017,0.000000e+00,-0.001979,-1.518631,1.334156,Monero,3
ETC,20,12,0.000115,2.100000e-04,-0.041907,-1.167016,1.012991,Ethereum Classic,3
ZEC,18,12,0.000007,2.100000e-05,-0.034542,-1.255502,1.092984,ZCash,3


In [24]:
# Plotting the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
alt.Chart(clustered_df).mark_circle().encode(
    alt.X('TotalCoinsMined', scale=alt.Scale(zero=False)),
    alt.Y('TotalCoinSupply', scale=alt.Scale(zero=False, padding=1)),
    color='class').configure_axis(grid=False).properties(title='Tradable Currencies Visualization', width=600)

alt.Chart(...)

In [ ]:
# Showing the table of current tradable cryptocurrencies
display(clustered_df)